Q. Using a deep learning algorithm of your choice, learn the representation of dark matter in the strong gravitational lensing images provided using PyTorch.
The goal is to use this learned representation for anomaly detection, hence you should pick the most appropriate algorithm and discuss your strategy.

A. Considering the purpose is anomaly detection, I choose Autoencoder as the most appropriate method, as discussed in [1]. I think Convolutional VAE (CVAE)[5] especially suits this purpose.
Convolutional architecture is known to function relatively better in tasks about images than MLP(multilayer perception).
Convolutional AutoEncoder(hereafter CAE)[2] incorpolates CNN's algorithm into itself, and according to [3],[4] (written in Japnanese), it clearly shows that CAE overwhelms other MLP algorithms, and it is said that it can function well without GPU.
CVAE evolved from CAE, which add features of VAE. it is implemented on purpose of 

[1]Deep Learning the Morphology of Dark Matter Substructure, pp8
https://arxiv.org/pdf/1909.07346.pdf

[2]Deep Clustering with Convolutional Autoencoders
https://link.springer.com/chapter/10.1007/978-3-319-70096-0_39

[3]PytorchによるAutoEncoder Familyの実装(Implementation of various Autoencoders using PyTorch)
https://elix-tech.github.io/ja/2016/07/17/autoencoder.html

[4]様々なオートエンコーダによる異常検知(Anomaly Detection with lots of Autoencoders)
https://sinyblog.com/deaplearning/auto_encoder_001/

[5]Semi-Supervised Learning with Deep Generative Models (CVAE)
https://arxiv.org/abs/1406.5298

[6]Latent representations of transient candidates from an astronomical image difference pipeline using Variational Autoencoders
https://pdfs.semanticscholar.org/57ce/a9520008706b8cb190d94513e695068210cd.pdf

[7]Convolutional Sparse Autoencoders for Image Classification (SCAE)
https://ieeexplore.ieee.org/document/7962256

In [1]:
import torch
import torchvision